# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.232763009617                   -0.50    8.0
  2   -7.250079594702       -1.76       -1.40    1.0
  3   -7.251098621046       -2.99       -1.98    2.0
  4   -7.251153099805       -4.26       -1.99    2.0
  5   -7.251323668291       -3.77       -2.54    1.0
  6   -7.251337883065       -4.85       -3.11    1.0
  7   -7.251338713509       -6.08       -3.58    2.0
  8   -7.251338788846       -7.12       -4.09    1.0
  9   -7.251338797562       -8.06       -4.74    2.0
 10   -7.251338798501       -9.03       -4.80    3.0
 11   -7.251338798656       -9.81       -5.16    1.0
 12   -7.251338798681      -10.61       -5.29    3.0
 13   -7.251338798700      -10.72       -5.69    1.0
 14   -7.251338798704      -11.38       -6.44    2.0
 15   -7.251338798704      -12.67       -6.49    2.0
 16   -7.251338798705      -13.01       -7.22    1.0
 17   -7.251338798705      -14.27       -7.24 

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04910362100310163
[ Info: Arnoldi iteration step 2: normres = 0.32685171711216576
[ Info: Arnoldi iteration step 3: normres = 0.5031957585647238
[ Info: Arnoldi iteration step 4: normres = 0.5079185939475834
[ Info: Arnoldi iteration step 5: normres = 0.23231998806838075
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (5.16e-02, 1.27e-02, 1.55e-01, 7.41e-02, 1.47e-01)
[ Info: Arnoldi iteration step 6: normres = 0.5053162890130384
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.17e-02, 4.61e-02, 4.93e-01, 7.43e-02, 1.69e-02)
[ Info: Arnoldi iteration step 7: normres = 0.17248495736530225
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.32e-03, 8.24e-02, 9.60e-03, 1.35e-01, 6.03e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11280525268200924
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.15e